<a href="https://colab.research.google.com/github/sudhirshahu51/RAG/blob/main/pdf_query_with_Hybrid_Search_and_cross_reranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index langchain chromadb pypdf llama-index-llms-groq llama-index-embeddings-huggingface llama-index-vector-stores-chroma --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.8 MB/s eta 0:00:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install llama-index-retrievers-bm25 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.7/646.7 kB 18.9 MB/s eta 0:00:00


In [ ]:
# to make async work well with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# **1. Text Extraction from PDF file**

In [ ]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
#documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Colab Notebooks/Data/microsoft_annual_report_2022.pdf"]).load_data()sudhirjk
documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Takeout/colab notebooks/microsoft_annual_report_2022.pdf"]).load_data()

In [ ]:
documents[5].text

'4 \n  \nOur commitment to responsibly develop and use technologies like AI is core to who we are. We put our commitment into \npractice, not only within Microsoft but by empowering our customers and partners to do the same and by advocating for \npolicy change. We released our Responsible AI Standard, which outlines 17 goals aligned to our six AI principles and \nincludes tools and practices to support them. And we share our open -source tools, including the new Responsible AI \nDashboard, to help developers building AI technologies identify and mitigate issues before deployment.  \nFinally, we provide clear reporting and information on how we run our business and how we work with customers and \npartners, delivering the transparency that is central to trust. Our annual Impact Summary shares more about our progress \nand learnings across these four commitments, and our Reports Hub  provides detailed reports on our environmental data, \nour political activities, our workforce demograph

In [ ]:
len(documents)

93

# **2. Splitting Chunks**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from llama_index.core.node_parser import LangchainNodeParser          #understand sentence and token based splitting can also be done iusing llamaindex or wrapper of langchain in llamaindex.

In [ ]:
parser  = LangchainNodeParser(RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=512,
    chunk_overlap=100))
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
len(nodes)

684

# **3. Create embeddings**

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
import pandas as pd

In [ ]:
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import chromadb
# chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction


In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_20221")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model, show_progress=True, use_async=True)


Generating embeddings: 100%|██████████| 69/69 [00:16<00:00,  4.14it/s]


In [ ]:
#index = VectorStoreIndex.from_documents(
#    documents, storage_context=storage_context, embed_model=embed_model, transformation = [SentenceSplitter(chunk_size=25, chunk_overlap=0),
#        TitleExtractor(),], show_progress=True, use_async=True
#)

# **4. Calling LLM using GROQ API**

In [ ]:
from google.colab import userdata
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage
groq_key = userdata.get('GROQ_API')
llm = Groq(model="llama3-70b-8192", api_key=groq_key) #https://docs.llamaindex.ai/en/stable/examples/llm/groq/#streaming

In [ ]:
response = llm.complete("Explain the importance of low latency LLMs")

In [ ]:
print(response)

Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Interactive Systems**: In interactive systems like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, ensuring a seamless user experience. Delayed responses can lead to user frustration and abandonment.
2. **Real-time Decision Making**: In applications like autonomous vehicles, medical diagnosis, or financial trading, low-latency LLMs can quickly process and analyze vast amounts of data to make timely decisions, which can be critical in life-or-death situations.
3. **Live Streaming and Broadcasting**: Low-latency LLMs can facilitate real-time language translation, sentiment analysis, or content moderation in live streaming and broadcasting, enabling a more engaging and responsive viewer experience.
4. **Gaming and Esports**: In online gaming 

In [ ]:
response = llm.stream_complete("Explain the importance of low latency LLMs")

In [ ]:

for r in response:
    print(r.delta, end="")

Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Interactive Systems**: In interactive systems like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, ensuring a seamless user experience. Delays in response can lead to user frustration and abandonment.
2. **Real-time Decision Making**: In applications like autonomous vehicles, medical diagnosis, or financial trading, low-latency LLMs can quickly process and analyze vast amounts of data to make timely decisions, which can be critical in life-or-death situations or high-stakes environments.
3. **Live Streaming and Broadcasting**: Low-latency LLMs can facilitate real-time language translation, subtitling, or closed captions for live events, such as conferences, sports, or news broadcasts, making them more accessible to a broader audience.
4. *

# **5. Query Engine**

### **i. BM25 Retriver**

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

In [ ]:
# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

DEBUG:bm25s:Building index from IDs objects


In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve("Measure revenue sources of mircrosoft?")

for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 70170506-e321-45b2-9c99-836740f19d07<br>**Similarity:** 2.5193252563476562<br>**Text:** impracticability of determining the geographic source of the revenue.  
Revenue, classified by significant product and service offerings, was as follows:  
  
(In millions)         
        
Year Ended June 30, 2022 2021 2020 
        
Server products and cloud services $ 67,321  $ 52,589  $ 41,379  
Office products and cloud services  44,862   39,872   35,316  
Windows  24,761   22,488   21,510  
Gaming  16,230   15,370   11,575  
LinkedIn  13,816   10,289   8,077<br>

**Node ID:** 1458c616-0bfa-42b8-84a9-fb972a375fbf<br>**Similarity:** 2.3889145851135254<br>**Text:** may incur future impairments.  
Equity investments with readily determinable fair values are measured at fair value. Equity investments without readily 
determinable fair values are measured using the equity method or measured at cost with adjustments for observable 
changes in price or impairments (referred to as the measurement alternative). We perform a qualitative assessment on a<br>

### **ii. Vector Store Retriever**

In [ ]:
vector_retriever  = index.as_retriever(similarity_top_k=2, )

In [ ]:
# Query Data
query_engine = index.as_query_engine(llm = llm)
response = query_engine.query("Measure revenue sources of mircrosoft?")
display(Markdown(f"<b>{response}</b>"))

<b>Microsoft's revenue sources can be measured by the following categories: Server products and cloud services, Office products and cloud services, Windows, Gaming, and LinkedIn. Additionally, product revenue includes sales from operating systems, cross-device productivity applications, server applications, business solution applications, desktop and server management tools, software development tools, video games, and hardware such as PCs, tablets, gaming and entertainment consoles, other intelligent devices, and related accessories.</b>

#**6. Hybrid Search**

In [ ]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

## **i. Hybrid Search using Reciprocal Rerank Fusion score.**

In [ ]:
from llama_index.core.retrievers import QueryFusionRetriever


retriever = QueryFusionRetriever(
        [vector_retriever,
        bm25_retriever],
        similarity_top_k=5, #Get top 5 nodes from naximum of 2*(5)(vector retriver) + 2*(5)(bm25) = 20 nodes. (5) for 5 artificial queries.
        num_queries = 5,  # 5 Artificial Query generation.
        use_async=True)

In [ ]:
# check how decide Alpha for Above

In [ ]:
nodes = retriever.retrieve("Measure revenue sources of mircrosoft?")

for node in nodes:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")
    display_source_node(node, source_length=5000)
    print("--------------------------------------------------")


Score: 6.42 - to access our content and services, and the ability to create new experiences through first-party content creators.  
Search and News Advertising  
Our Search and news advertising business is designed to deliver relevant search, native, and display advertising to a 
global audience. We have several partnerships with other companies, including Yahoo, through which we provide and 
monetize search queries. Growth depends on our ability to attract new users, understand intent, and match intent with...
-----



**Node ID:** 6f3148fc-ff12-4b6f-a641-0b4f1e1a608d<br>**Similarity:** 6.423602104187012<br>**Text:** to access our content and services, and the ability to create new experiences through first-party content creators.  
Search and News Advertising  
Our Search and news advertising business is designed to deliver relevant search, native, and display advertising to a 
global audience. We have several partnerships with other companies, including Yahoo, through which we provide and 
monetize search queries. Growth depends on our ability to attract new users, understand intent, and match intent with<br>

--------------------------------------------------
Score: 4.58 - web.queries@computershare.com  
You can also send mail to the transfer agent at:  
Computershare  
P.O. Box 43006  
Providence RI 02940-3078  
Shareholders can sign up for electronic alerts to access the 
annual report and proxy statement online. The service gets 
you the information you need faster and also gives you the 
power and convenience of online proxy voting. To sign up for 
this free service, visit the Annual Report site on the Investor 
Relations website at:...
-----



**Node ID:** 4973dafa-e242-4854-95b6-acee7ae88bb9<br>**Similarity:** 4.581233501434326<br>**Text:** web.queries@computershare.com  
You can also send mail to the transfer agent at:  
Computershare  
P.O. Box 43006  
Providence RI 02940-3078  
Shareholders can sign up for electronic alerts to access the 
annual report and proxy statement online. The service gets 
you the information you need faster and also gives you the 
power and convenience of online proxy voting. To sign up for 
this free service, visit the Annual Report site on the Investor 
Relations website at:<br>

--------------------------------------------------
Score: 3.99 - 84 
REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOUNTING FIRM  
To the Stockholders and the Board of Directors of Microsoft Corporation  
Opinion on the Financial Statements  
We have audited the accompanying consolidated balance sheets of Microsoft Corporation and subsidiaries (the 
“Company”) as of June 30, 2022 and 2021, the related consolidated statements of income, comprehensive income, cash...
-----



...

--------------------------------------------------
Score: 3.71 - across the Company. 
In addition to our main research and development operations, we also operate Microsoft Research. Microsoft Research is 
one of the world’s largest corporate research organizations and works in close collaboration with top universities around 
the world to advance the state-of-the-art in computer science and a broad range of other disciplines, providing us a unique 
perspective on future trends and contributing to our innovation....
-----



**Node ID:** f6a8a3c8-3800-49d1-9dd3-c6ae26aefa93<br>**Similarity:** 3.7086267471313477<br>**Text:** across the Company. 
In addition to our main research and development operations, we also operate Microsoft Research. Microsoft Research is 
one of the world’s largest corporate research organizations and works in close collaboration with top universities around 
the world to advance the state-of-the-art in computer science and a broad range of other disciplines, providing us a unique 
perspective on future trends and contributing to our innovation.<br>

--------------------------------------------------
Score: 2.52 - impracticability of determining the geographic source of the revenue.  
Revenue, classified by significant product and service offerings, was as follows:  
  
(In millions)         
        
Year Ended June 30, 2022 2021 2020 
        
Server products and cloud services $ 67,321  $ 52,589  $ 41,379  
Office products and cloud services  44,862   39,872   35,316  
Windows  24,761   22,488   21,510  
Gaming  16,230   15,370   11,575  
LinkedIn  13,816   10,289   8,077...
-----



**Node ID:** 70170506-e321-45b2-9c99-836740f19d07<br>**Similarity:** 2.5193252563476562<br>**Text:** impracticability of determining the geographic source of the revenue.  
Revenue, classified by significant product and service offerings, was as follows:  
  
(In millions)         
        
Year Ended June 30, 2022 2021 2020 
        
Server products and cloud services $ 67,321  $ 52,589  $ 41,379  
Office products and cloud services  44,862   39,872   35,316  
Windows  24,761   22,488   21,510  
Gaming  16,230   15,370   11,575  
LinkedIn  13,816   10,289   8,077<br>

--------------------------------------------------


In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever)

In [ ]:
response = query_engine.query("Measure revenue sources of mircrosoft?")
display(Markdown(f"<b>{response}</b>"))

<b>Microsoft's revenue sources can be measured by the following significant product and service offerings: Server products and cloud services, Office products and cloud services, Windows, Gaming, and LinkedIn.</b>

## **ii\. Hybrid Search with query expansion and using cosine similarity for final nodes**

# Cross- ReRanker

# **Query Expansion**

In [ ]:
!pip install umap-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.6 MB/s eta 0:00:00


In [ ]:
import umap.umap_ as umap
import numpy as np
from tqdm import tqdm


*   Expansion with multiple queries



In [ ]:
def augment_multiple_query(query, llm):

    messages = [
    ChatMessage(
        role="system", content="You are a helpful expert financial research assistant. Your users are asking questions about an annual report. "
            "Suggest up to five additional related questions to help them find the information they need, for the provided question. "
            "Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic."
            "Make sure they are complete questions, and that they are related to the original question."
            "Output one question per line. Do not number the questions."
    ),
    ChatMessage(role="user", content=query ),
    ]
    resp = llm.chat(messages)
    return resp

In [ ]:
original_query = "What were the most important factors that contributed to increases in revenue?"
aug_queries = augment_multiple_query(original_query, llm)
print(aug_queries)


assistant: What was the total revenue for the year?
Which product or service segment generated the most revenue?
Were there any significant changes in pricing or sales volume?
Did the company experience any notable acquisitions or divestitures?
Were there any changes in the company's business model or strategy?


In [ ]:
augmented_queries = aug_queries.message.blocks[0].text.split("\n")
queries = [original_query] + augmented_queries

In [ ]:
vector_retriever  = index.as_retriever(similarity_top_k=2,)
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

DEBUG:bm25s:Building index from IDs objects


In [ ]:
all_nodes = []

In [ ]:
for query in queries:
  nodes_v = vector_retriever.retrieve(original_query)
  nodes_b = bm25_retriever.retrieve(original_query)
  all_nodes = all_nodes + nodes_v + nodes_b

In [ ]:
unique_nodes = {}
for nod in all_nodes:
  if nod.node_id not in unique_nodes.keys():unique_nodes[nod.node_id] = nod.text

In [ ]:
 node_df = pd.DataFrame({'node_id': unique_nodes.keys(), 'text': unique_nodes.values()}, index = range(len(unique_nodes)))

In [ ]:
# Query Data
query_engine = index.as_query_engine(llm = llm)
response = query_engine.query("Measure revenue sources of mircrosoft?")
display(Markdown(f"<b>{response}</b>"))

<b>Microsoft's revenue sources can be measured by the following categories: Server products and cloud services, Office products and cloud services, Windows, Gaming, and LinkedIn. Additionally, Product revenue includes sales from operating systems, cross-device productivity applications, server applications, business solution applications, desktop and server management tools, software development tools, video games, and hardware such as PCs, tablets, gaming and entertainment consoles, other intelligent devices, and related accessories.</b>

1. Take 20 chunks using vector retriver for augumneted queries.
2. Take 20 chunks using BM25 retriver for augumneted queries.
3. remove the duplicates.
4.  Based u[on dot matrix product with query, choose top 5 and pass as context.

Now integrate re-ranker, graph search too and later on image search.  

In [ ]:
node_df

,node_id,text
0,253d6dd6-f2d4-40da-ae0b-6f00fecf8f59,• Windows revenue increased $2.3 billion or 1...
1,7d606683-7718-43a7-9411-d52ad4f6faec,Cost of revenue increased $10.4 billion or 20%...
2,7d70e72b-8d2e-4f00-b6e7-229231142aea,funded retirement benefits for all plans were ...
3,baa1dd64-2e3d-4b36-9a16-903bacc5a32d,Savings Plans \nWe have savings plans in the ...


# **6. Re-ranking**

In [ ]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
node_df['original_query'] = original_query

In [ ]:
pairs = node_df[['original_query', 'text']].values.tolist()

In [ ]:
scores = cross_encoder.predict(pairs) #learn how cross encoders works

In [ ]:
node_df['scores'] = scores

In [ ]:
top_chunks = node_df.nlargest(5, ['scores'])['text'].tolist()

Call LLM and use prompt template to pass conetxt and get the response and Also try reranker from llamindex directly and call llm using that.

In [ ]:
def rag(query, retrieved_documents, llm):
    information = "\n\n".join(retrieved_documents)

    messages = [
    ChatMessage(
        role="system", content="You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report. You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
    ),
    ChatMessage(role="user", content=f"Question: {query}. \n Information: {information}"),
    ]

    resp = llm.stream_chat(messages)
    return resp


In [ ]:
output = rag(query=original_query, retrieved_documents=top_chunks, llm = llm)

for r in output:
    print(r.delta, end="")

According to the annual report, the most important factors that contributed to increases in revenue were:

1. Growth in Windows OEM revenue, driven by continued strength in the commercial PC market, which has higher revenue per license.
2. Growth in Windows Commercial products and cloud services revenue, driven by demand for Microsoft 365.

These two factors contributed to a 10% increase in Windows revenue, which was a significant driver of overall revenue growth.

Now Use Evaluation and then Integrate Graph Search.

# **LLM Evaluation**

# **7. Apply Embedding Adaptors to improve the results more.**
from 5th tutorial.

# **8. Use Graph Search**

[link](https://superlinked.com/vectorhub/articles/optimizing-rag-with-hybrid-search-reranking)